In [1]:
import numpy as np                   
import pandas as pa                 
import matplotlib.pyplot as plt      
import seaborn as sn                
import numpy.random as rn
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LogisticRegression
import matplotlib.colors as pltco 
from sklearn.model_selection import cross_val_score

### Import Data Frame assign the name fifa
* Print out the head() or first 5 rows

In [2]:
fifa = pa.read_csv('https://www.dropbox.com/s/ov362sryogd0942/CompleteDataset.csv?dl=1', low_memory=False)

fifa.head()

,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,Acceleration,Aggression,...,Short passing,Shot power,Sliding tackle,Sprint speed,Stamina,Standing tackle,Strength,Vision,Volleys,Position
0,Cristiano Ronaldo,32,Portugal,94,94,Real Madrid CF,€95.5M,€565K,89,63,...,83,94,23,91,92,31,80,85,88,ST LW
1,L. Messi,30,Argentina,93,93,FC Barcelona,€105M,€565K,92,48,...,88,85,26,87,73,28,59,90,85,RW
2,Neymar,25,Brazil,92,94,Paris Saint-Germain,€123M,€280K,94,56,...,81,80,33,90,78,24,53,80,83,LW
3,L. Suárez,30,Uruguay,92,92,FC Barcelona,€97M,€510K,88,78,...,83,87,38,77,89,45,80,84,88,ST
4,M. Neuer,31,Germany,92,92,FC Bayern Munich,€61M,€230K,58,29,...,55,25,11,61,44,10,83,70,11,GK


### Fixing Bad Data
* Stripped "€" and "K" from Wage
* Stripped "+" and "-" from the front of the columns
* Removed any part of the String after "+" and "-" and convert columns to float

In [3]:
fifa['Wage'] = fifa['Wage'].map(lambda x: x.lstrip('€').rstrip('K'))

In [4]:
A = fifa["Age"] = fifa.Age.astype(float)
B = fifa["Overall"] = fifa.Overall.astype(float)
C = fifa["Wage"] = fifa.Wage.astype(float)
D = fifa["Potential"] = fifa.Potential.astype(float)

In [5]:
def removeStrings (X):
    if '+' in X:
        K = X.find('+')
        return X[:K]
    if '-' in X:
        K = X.find('-')
        return X[:K]
    else:  
        return X

In [6]:
def convertFloat(fifa, feature):
    for K in fifa.index: 
        fifa.loc[K,feature] = removeStrings(fifa.loc[K, feature])

    
    fifa.loc[:, feature] = fifa.loc[:,feature].astype("float")  
    return fifa

In [7]:
fifa = convertFloat(fifa, "Acceleration")
fifa = convertFloat(fifa, "Aggression")
fifa = convertFloat(fifa, "Agility")
fifa = convertFloat(fifa, "Balance")
fifa = convertFloat(fifa, "Ball control")

### Avalaible Columns
* Age, Overall, Potentail, Wage, Acceleration, Aggression, Agility, Balance, and Ball Control

## K- nearest neighbors

In [8]:
X = np.array(fifa[['Ball control', 'Overall']])
y = np.array(fifa.iloc[:, 7])

In [9]:
for l in range(X.shape[1]):
    X[:, l] = ( X[:, l] - min(X[:, l]))/(max(X[:, l]) - min(X[:, l]))

In [10]:
n = X.shape[0]
test = int(0.30*n)          
perm = rn.permutation(n)   
X = X[perm]
y = y[perm]
X_test = X[:test]      
y_test = y[:test]
X_train = X[test:]   
y_train = y[test:]

In [11]:
def plot_reg_model(reg, X_train, y_train, X_test, y_test):
    
    h = 0.01 
    
    x_min, x_max = -0.1, 1.1
    y_min, y_max = -0.1, 1.1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
        np.arange(y_min, y_max, h))
    Z = reg.predict(np.c_[xx.ravel(), yy.ravel()])

    Z = Z.reshape(xx.shape)

    plt.figure(figsize=(8, 6))
    plt.contour(xx, yy, Z)
    
    plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, marker='o')
    plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, marker='+');
    
    print('Training R2: {}'.format(reg.score(X_train, y_train)))
    print('Testing R2: {}'.format(reg.score(X_test, y_test)))

In [12]:
reg = KNeighborsRegressor(n_neighbors=100)
reg.fit(X_train, y_train)
plot_reg_model(reg, X_train, y_train, X_test, y_test)

Training R2: 0.6592982666287416
Testing R2: 0.6680886178687546


#### Cross Validation

In [13]:
knn = cross_val_score(reg, X, y, cv=20)
print('KNN Mean R2: {}'.format(knn.mean()))
print('KNN Standard Deviation R2: {}'.format(knn.std()))

KNN Mean R2: 0.6814245284708472
KNN Standard Deviation R2: 0.0423092817477741


In [14]:
X = np.array(fifa[['Ball control', 'Agility']])
y = np.array(fifa.iloc[:, 3])

In [15]:
for l in range(X.shape[1]):
    X[:, l] = ( X[:, l] - min(X[:, l]))/(max(X[:, l]) - min(X[:, l]))

In [16]:
n = X.shape[0]
test = int(0.30*n)          
perm = rn.permutation(n)   
X = X[perm]
y = y[perm]
X_test = X[:test]       
y_test = y[:test]
X_train = X[test:]     
y_train = y[test:]

In [17]:
reg1 = KNeighborsRegressor(n_neighbors=25)
reg1.fit(X_train, y_train)
plot_reg_model(reg1, X_train, y_train, X_test, y_test)

Training R2: 0.5417829584536835
Testing R2: 0.5119058759638677


#### Cross Validation

In [18]:
knn = cross_val_score(reg1, X, y, cv=20)
print('KNN Mean R2: {}'.format(knn.mean()))
print('KNN Standard Deviation R2: {}'.format(knn.std()))

KNN Mean R2: 0.5056440515288595
KNN Standard Deviation R2: 0.0230497785563941


## PCA Model
* PCA Model with colums 7,8,9 vs Age

In [19]:
X = np.array(fifa.iloc[:,7:10])
y = np.array(fifa.iloc[:, 1])

pca = PCA(n_components=2)
Xpca = pca.fit_transform(X);

import numpy.random as rn
n = X.shape[0]
test = int(0.25*n)          
perm = rn.permutation(n)   
X = Xpca[perm]
y = y[perm]
X_test = Xpca[:test]     
y_test = y[:test]
X_train = Xpca[test:]   
y_train = y[test:]


In [20]:
cat1 = LogisticRegression(solver='liblinear', multi_class='ovr').fit(X_train[:, 0:2], y_train)
cat1.score(X_train[:, 0:2], y_train), cat1.score(X_test[:, 0:2], y_test)

(0.083197389885807507, 0.08097886540600667)

In [21]:
def plot_reg_model(reg, X_train, y_train, X_test, y_test):

    h = 0.01 
    
    x_min, x_max = -40.0, 40.0
    y_min, y_max = -20.0, 20.0
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
        np.arange(y_min, y_max, h))
    Z = reg.predict(np.c_[xx.ravel(), yy.ravel()])

    Z = Z.reshape(xx.shape)

    plt.figure(figsize=(8, 6))
    plt.contour(xx, yy, Z)
    
    plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, marker='o')
    plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, marker='+');
    
    print('Training R2: {}'.format(reg.score(X_train, y_train)))
    print('Testing R2: {}'.format(reg.score(X_test, y_test)))

In [22]:
reg = LinearRegression()
reg.fit(X_train,y_train)
plot_reg_model(reg, X_train, y_train, X_test, y_test)

Training R2: 9.737843962831505e-05
Testing R2: -0.0073887372375158735


#### Cross Validation

In [23]:
pca = cross_val_score(reg, X, y, cv=10)
print('PCA Mean R2: {}'.format(pca.mean()))
print('PCA Standard Deviation R2: {}'.format(pca.std()))

PCA Mean R2: 0.03860141727167922
PCA Standard Deviation R2: 0.0056116538256946276


* PCA Model With Columns 10,11,12 vs Age

In [24]:
X = np.array(fifa.iloc[:,10:13])                        
y = np.array(fifa.iloc[:, 1])

pca = PCA(n_components=2) 
Xpca = pca.fit_transform(X);

import numpy.random as rn
n = X.shape[0]
test = int(0.25*n)            
perm = rn.permutation(n)   
X = Xpca[perm]
y = y[perm]
X_test = Xpca[:test]      
y_test = y[:test]
X_train = Xpca[test:]    
y_train = y[test:]


In [25]:
cat2 = LogisticRegression(solver='liblinear', multi_class='ovr').fit(X_train[:, 0:2], y_train)
cat2.score(X_train[:, 0:2], y_train), cat1.score(X_test[:, 0:2], y_test)

(0.079860596173809875, 0.088320355951056737)

In [26]:
reg2 = LinearRegression()
reg2.fit(X_train,y_train)
plot_reg_model(reg2, X_train, y_train, X_test, y_test)

Training R2: 6.775209568654628e-05
Testing R2: -0.0017958303912648521


#### Cross Validation

In [27]:
pca = cross_val_score(reg2, X, y, cv=10)
print('PCA Mean R2: {}'.format(pca.mean()))
print('PCA Standard Deviation R2: {}'.format(pca.std()))

PCA Mean R2: 0.033635783816
PCA Standard Deviation R2: 0.0066551921192006


### Regression Models Liner, Ridge, and Lasso

In [28]:
X = np.array(fifa[['Age', 'Overall']])
y = np.array(fifa['Wage'])

In [29]:
def plot_reg_model(reg, X, y):


    n = X.shape[0]
    test = int(0.20*n)           
    perm = rn.permutation(n)   
    X = X[perm]
    y = y[perm]
    X_test = X[:test]      
    y_test = y[:test]
    X_train = X[test:]     
    y_train = y[test:]
    
    reg.fit(X_train, y_train) 
     
    nn = 200
    
    x_min, x_max = X[:, 0].min()-0.1, X[:, 0].max()+0.1
    y_min, y_max = X[:, 1].min()-0.1, X[:, 1].max()+0.1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, nn),   
        np.linspace(y_min, y_max, nn))                    
    Z = reg.predict(np.c_[xx.ravel(), yy.ravel()]) 

    Z = Z.reshape(xx.shape)

    plt.figure(figsize=(8, 6))
    plt.contour(xx, yy, Z)
    
    plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, marker='o')
    plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, marker='+');
    
    print('Training R2: {}'.format(reg.score(X_train, y_train)))
    print('Testing R2: {}'.format(reg.score(X_test, y_test)))
    
    return reg

In [30]:
reg = LinearRegression()
plot_reg_model(reg, X, y)

Training R2: 0.3669833757909776
Testing R2: 0.3932160534635045


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [31]:
ridge_reg = Ridge(alpha=2)
plot_reg_model(ridge_reg, X, y)

Training R2: 0.3758218833275556
Testing R2: 0.35706680010006386


Ridge(alpha=2, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [32]:
lasso_reg=Lasso(alpha=1)
plot_reg_model(lasso_reg, X, y)

Training R2: 0.36897094940994823
Testing R2: 0.3827030598064625


Lasso(alpha=1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [33]:
knn = KNeighborsRegressor(n_neighbors=10)
plot_reg_model(knn, X, y)

Training R2: 0.7619530068554001
Testing R2: 0.7196023838521335


KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=10, p=2,
          weights='uniform')

In [34]:
tree = DecisionTreeRegressor(min_samples_leaf=5)
plot_reg_model(tree, X, y)

Training R2: 0.7737315418578755
Testing R2: 0.7283026490989915


DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=5, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best')

#### Cross Validation

In [59]:
lreg = cross_val_score(reg, X, y, cv=10)
ridge = cross_val_score(ridge_reg, X, y, cv=10)
LASSO = cross_val_score(lasso_reg, X, y, cv=10)
TREE = cross_val_score(tree, X, y, cv=10)

TypeError: estimator should be an estimator implementing 'fit' method, array([-0.31919328, -0.17248263, -0.09630928, -0.04777462, -0.25624203,
       -0.07953845, -0.14363067, -0.09929756, -1.92646246, -0.28579747]) was passed

In [ ]:
print('Linear Regression Mean R2: {}'.format(lreg.mean()))
print('Linear Regression Standard Deviation R2: {}'.format(lreg.std()))

In [36]:
print('Ridge Mean R2: {}'.format(ridge.mean()))
print('Ridge Standard Deviation R2: {}'.format(ridge.std()))

Ridge Mean R2: -83.31767530533004
Ridge Standard Deviation R2: 237.0169155046805


In [37]:
print('Lasso Mean R2: {}'.format(LASSO.mean()))
print('Lasso Standard Deviation R2: {}'.format(LASSO.std()))

Lasso Mean R2: -81.03511768363349
Lasso Standard Deviation R2: 230.99397781248348


In [38]:
print('Tree Mean R2: {}'.format(TREE.mean()))
print('Tree Standard Deviation R2: {}'.format(TREE.std()))

Tree Mean R2: -0.09281349373616797
Tree Standard Deviation R2: 0.093042535633659


## Nueral Networks

In [39]:
X = np.array(fifa[['Agility', 'Overall']])
y = np.array(fifa['Wage'])

In [40]:
def plot_reg_model(reg, X, y):


    n = X.shape[0]
    test = int(0.20*n)          
    perm = rn.permutation(n)   
    X = X[perm]
    y = y[perm]
    X_test = X[:test]     
    y_test = y[:test]
    X_train = X[test:]   
    y_train = y[test:]
    
    reg.fit(X_train, y_train) 
     
    nn = 200
   
    x_min, x_max = X[:, 0].min()-0.1, X[:, 0].max()+0.1
    y_min, y_max = X[:, 1].min()-0.1, X[:, 1].max()+0.1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, nn),   
        np.linspace(y_min, y_max, nn))                    
    Z = reg.predict(np.c_[xx.ravel(), yy.ravel()]) 
   
    Z = Z.reshape(xx.shape)

    plt.figure(figsize=(8, 6))
    plt.contour(xx, yy, Z)
    
    plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, marker='o')
    plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, marker='+');
    
    print('Training R2: {}'.format(reg.score(X_train, y_train)))
    print('Testing R2: {}'.format(reg.score(X_test, y_test)))
    
    return reg

### Nueral Network Ridge Regression

In [41]:
r_reg = Ridge(alpha=2)
plot_reg_model(r_reg, X, y);
plt.savefig('15.2.png')

Training R2: 0.35474001383926657
Testing R2: 0.3453986917759261


In [42]:
ridge = cross_val_score(r_reg, X, y, cv=10)
print('Ridge Mean R2: {}'.format(ridge.mean()))
print('Ridge Standard Deviation R2: {}'.format(ridge.std()))

Ridge Mean R2: -89.69855976614755
Ridge Standard Deviation R2: 257.05798743335185


### Nueral Network Lasso Regression

In [43]:
l_reg=Lasso(alpha=1)
plot_reg_model(l_reg, X, y);
plt.savefig('15.3.png')

Training R2: 0.35631975384252834
Testing R2: 0.340214996265659


In [44]:
lasso = cross_val_score(l_reg, X, y, cv=10)
print('Lasso Mean R2: {}'.format(lasso.mean()))
print('Lasso Standard Deviation R2: {}'.format(lasso.std()))

Lasso Mean R2: -86.86846732479378
Lasso Standard Deviation R2: 248.97339510556557


### Nueral Network K-nearest Neighbors Regression

In [45]:
knn = KNeighborsRegressor(n_neighbors=6)
plot_reg_model(knn, X, y);
plt.savefig('15.4.png')

Training R2: 0.7673038560017927
Testing R2: 0.7289700578452859


In [46]:
knn = cross_val_score(knn, X, y, cv=10)
print('KNN Mean R2: {}'.format(knn.mean()))
print('KNN Standard Deviation R2: {}'.format(knn.std()))

KNN Mean R2: -0.2503245935877572
KNN Standard Deviation R2: 0.11359501797490984


### Nueral Network Decision Tree Regression

In [47]:
tree = DecisionTreeRegressor(max_depth=10)
plot_reg_model(tree, X, y);

Training R2: 0.8276996467672159
Testing R2: 0.7059297012291045


In [48]:
tree = cross_val_score(tree, X, y, cv=10)
print('Tree Mean R2: {}'.format(tree.mean()))
print('Tree Standard Deviation R2: {}'.format(tree.std()))

Tree Mean R2: -0.34267284444441587
Tree Standard Deviation R2: 0.5351971245281321


### Ensemble Methods

In [49]:
forest = RandomForestRegressor(n_estimators=100, max_depth=3, n_jobs=-1)
plot_reg_model(forest, X, y);
plt.savefig('15.6.png')

Training R2: 0.7551948310029237
Testing R2: 0.7330168650242256


In [50]:
Ensemble = cross_val_score(forest, X, y, cv=10)
print('Ensemble Mean R2: {}'.format(Ensemble.mean()))
print('Ensemble Standard Deviation R2: {}'.format(Ensemble.std()))

Ensemble Mean R2: -0.5731684135549064
Ensemble Standard Deviation R2: 0.9784218831243557


### Gradient Boosting Esamble Network Model

In [51]:
boost = GradientBoostingRegressor(n_estimators=50, learning_rate=0.1, max_depth=5)
plot_reg_model(boost, X, y);
plt.savefig('15.7.png')

Training R2: 0.8202534675627411
Testing R2: 0.700207913819644


In [52]:
Boost = cross_val_score(boost, X, y, cv=10)
print('Boost Mean R2: {}'.format(Boost.mean()))
print('Boost Standard Deviation R2: {}'.format(Boost.std()))

Boost Mean R2: -0.10927113584264103
Boost Standard Deviation R2: 0.1313927757003617


### Nueral Networks for Regression

In [53]:
X = np.array(fifa[['Age', 'Overall']])
y = np.array(fifa['Wage'])

In [54]:
v = [0]*X.shape[1]
for k in range(X.shape[1]):
    M = X[:, k].max()
    m = X[:, k].min()
    v[k] = (X[:, k] - m)/(M - m)
Xn = np.c_[v].transpose()  

In [55]:
mlp = MLPRegressor(hidden_layer_sizes = (100), solver = 'lbfgs', activation = 'relu', alpha=0.0001)
plot_reg_model(mlp, Xn, y);
plt.savefig('15.8.png')

Training R2: 0.7666585949632214
Testing R2: 0.7005058249197871


In [56]:
mlp = cross_val_score(mlp, X, y, cv=10)
print('MLP Mean R2: {}'.format(mlp.mean()))
print('MLP Standard Deviation R2: {}'.format(mlp.std()))

MLP Mean R2: -77.00279422415073
MLP Standard Deviation R2: 216.51213390123678


* adding an addtional layer to improve performance

In [62]:
mlp2 = MLPRegressor(hidden_layer_sizes = (100, 100), solver = 'lbfgs', activation = 'relu', alpha=0.0001)
plot_reg_model(mlp2, Xn, y);
plt.savefig('15.9.png')

Training R2: 0.7524181447021974
Testing R2: 0.7569480223346262


In [65]:
mlp2 = MLPRegressor(hidden_layer_sizes = (100, 100), solver = 'lbfgs', activation = 'relu', alpha=0.0001)
mlp = cross_val_score(mlp2, Xn, y, cv=4)
print('MLP Mean R2: {}'.format(mlp.mean()))
print('MLP Standard Deviation R2: {}'.format(mlp.std()))

MLP Mean R2: -0.12631779034076399
MLP Standard Deviation R2: 0.2317989720036636


#### We tried to use the new method of running the cross validation on each our models but we are getting means and standard deviations that do not make sense and are completely off from our testing and training sets. Knowing that, we will try using different methods of cross validation from the scikitlearn website to see if those will work. If those different methods do not work we will go back to computing it by hand. 

## What's next
    * Input the decision tree (Jordana)
    * Find different method of cross validation that will produce accurate values (Lee)
    * Implement newest models (Lee and Luke)
    * Run already working models against different features (Linear,Ridge, Lasso - SAM) 
        (Remainding models will be split among Lee,Jordana, and Luke)